In [1]:
!pip3 install ipython-autotime
%load_ext autotime


import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

from termcolor import colored
import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# nb_workers=10
# my_dir='/content/drive/MyDrive/Next_Door/'
import re
from datetime import datetime
import pickle
import ast

from collections import Counter

import spacy
from spacy.tokens import Doc, Token, Span

nlp = spacy.load("en_core_web_sm")

def tags_to_counter(tags_list_x):
  tags_list_x = tags_list_x.split(',')
  tags_list_x = Counter(tags_list_x)
  return tags_list_x


def top_k_percent_keys(counter,k):
    total_items = len(counter)
    threshold = int(total_items * k)
    sorted_counter = counter.most_common()
    top_keys = []
    count = 0
    for key, value in sorted_counter:
        count += 1
        top_keys.append(key)
        if count >= threshold:
            break
    return top_keys

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


def get_subject_np(sentence):





  # Tokenize the sentence
  tokens = nltk.word_tokenize(sentence)

  # Use NLTK's part-of-speech (POS) tagger to tag the tokens
  tagged_tokens = nltk.pos_tag(tokens)

  # Use a chunk parser to extract the subject noun phrase
  grammar = r"""
      NP: {<DT|PRP\$>?<JJ>*<NN.*>+} # chunk sequences of DT, JJ, NN
      """
  cp = nltk.RegexpParser(grammar)
  tree = cp.parse(tagged_tokens)

  # Traverse the tree to extract the subject noun phrase
  for subtree in tree.subtrees():
      if subtree.label() == "NP":
          subject_np = " ".join(word for word, tag in subtree.leaves())
          break

  return subject_np

def get_subject_np(text):
    
    if re.search('There (should|must|has to be|needs to be|have to be)', text):
      text = re.sub('(?i)(there)?\s?(should|must|has|needs|have)\s?(to)?\s?(be)?',' ',text)
      return text
    doc = nlp(text)
    subject = ''
    for token in doc:
        if token.dep_ == 'nsubj' or token.dep_ == 'csubj':
            for child in token.subtree:
                if child.pos_ != 'PUNCT':
                    subject += ' ' + child.text
            return subject.strip()
        elif token.dep_ == 'ROOT':
            for child in token.subtree:
                if child.pos_ == 'NOUN':
                    subject += ' ' + child.text
                    for subchild in child.subtree:
                        if subchild.pos_ == 'ADJ':
                            subject += ' ' + subchild.text
                    return subject.strip()
    if not subject:
        for chunk in doc.noun_chunks:
            subject += ' ' + chunk.text
        return subject.strip()
    return ''

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


time: 16.2 s (started: 2023-05-24 18:57:47 +00:00)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 18.6 s (started: 2023-05-24 19:00:18 +00:00)


# Question Mark Arguments

In [ ]:
models = ['text-curie-001','text-babbage-001','text-davinci-001','text-davinci-002','text-davinci-003','gpt-3.5-turbo']
query_type = 'questionmark'
# query_type = 'proscons'
model = 'gpt-3.5-turbo'
# model = 'BingAI'
# model = 'text-davinci-002'
meta_data_df = pd.read_excel('.../Datasets/Meta-Data/Kialo_MetaData_popular.xlsx')

query_file_path = f'.../Datasets/Query-Results/Kialo - Open AI/kialo_{query_type}_{model}.csv'
# query_file_path = f'.../Datasets/Query-Results/Kialo - Bing AI/Bing AI Queries Merged - corrected.xlsx'

answers_df = pd.read_csv(query_file_path)
# answers_df = pd.read_excel(query_file_path)

answers_df['tags_list_translated']=meta_data_df['tags_list_translated'].copy()



time: 2.4 s (started: 2023-05-22 17:12:16 +00:00)


In [ ]:
answers_df

,post_id,url,initial_claim,model,query,answer,created_at,tags_list_translated
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Women have the right to control thei...,1684680193,"feminism,abortion,health,religion,women,law"
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Encourages creativity and self-expre...,1684680224,"art,government,economics,education"
2,9226,https://www.kialo.com/global-overpopulation-is...,Global overpopulation is a myth.,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. It challenges the commonly held beli...,1684680253,"society,humans,earth,environment,climatechange"
3,1266,https://www.kialo.com/gender-neutral-bathrooms...,All public bathrooms should be gender neutral.,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Inclusivity: Gender-neutral bathroom...,1684680278,"gender,feminism,transgender,sex,equality"
4,9647,https://www.kialo.com/should-homeschooling-be-...,Homeschooling should be banned.,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Homeschooling may not provide childr...,1684680305,"education,policy,usa,law,school,america"
...,...,...,...,...,...,...,...,...
2768,54485,https://www.kialo.com/ar-verta-vakcinuotis-nuo...,Nuo COVID-19 vakcinuotis verta,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Protection against COVID-19: Vaccina...,1684775364,"covid-19,vaccines"
2769,54924,https://www.kialo.com/%C2%BFson-seguros-los-mo...,"¿Son seguros los monopatines, patines eléctric...",gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n- Son una alternativa ecológica y econó...,1684775392,"transport,youth,mobility,sustainability,securi..."
2770,54934,https://www.kialo.com/schools-should-compete-f...,Should academic and educational institutions a...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Encourages institutions to maintain ...,1684775417,"academic,schools,university,public,schooling,e..."
2771,55046,https://www.kialo.com/discipline-or-abuse-when...,Discipline should be considered abuse only whe...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. It helps to establish clear boundari...,1684775443,children


time: 43.6 ms (started: 2023-05-22 17:12:27 +00:00)


In [ ]:
meta_data_df[meta_data_df['tags_list_translated'].str.contains('econom') & (meta_data_df['language']=='en')]

,post_id,title,url,last_edit_date,found_where,initial_claim,tags_list,language,claim_translation,tags_list_translated
1,9563,Should the government provide funding for arts...,https://www.kialo.com/should-the-government-pr...,2023-02-25T19:29:52.672Z,https://www.kialo.com/explore/popular,The government should provide funding for arts...,"Art,Government,Economics,Education",en,The government should provide funding for arts...,"art,government,economics,education"
7,5904,Should governments make an effort to reduce th...,https://www.kialo.com/should-governments-make-...,2023-03-15T12:38:22.544Z,https://www.kialo.com/explore/popular,Governments should make an effort to reduce th...,"Politics,Economics,Gender",en,Governments should make an effort to reduce th...,"politics,economics,gender"
13,14053,Should governments provide a universal basic i...,https://www.kialo.com/should-governments-provi...,2022-10-03T12:58:33.203Z,https://www.kialo.com/explore/popular,Governments should provide a universal basic i...,"UBI,Economics,Politics,Society,Equality",en,Governments should provide a universal basic i...,"ubi,economics,politics,society,equality"
32,1634,Should There be a Universal Basic Income (UBI)?,https://www.kialo.com/should-there-be-a-univer...,2023-03-08T08:08:42.899Z,https://www.kialo.com/explore/popular,Wealthy countries should provide citizens with...,"Economics,Equality,Money,Government,Politics,P...",en,Wealthy countries should provide citizens with...,"economics,equality,money,government,politics,p..."
45,9326,Should nuclear energy replace fossil fuels?,https://www.kialo.com/should-nuclear-energy-re...,2023-03-14T14:37:51.995Z,https://www.kialo.com/explore/popular,Nuclear energy should be used to replace fossi...,"Science,Environment,Economics,ClimateChange",en,Nuclear energy should be used to replace fossi...,"science,environment,economics,climatechange"
...,...,...,...,...,...,...,...,...,...,...
2644,49892,Should internet infrastructure be publicly owned?,https://www.kialo.com/should-internet-infrastr...,2021-05-27T09:07:43.161Z,https://www.kialo.com/explore/popular,Internet infrastructure should be publicly owned.,"Internet,Technology,Economics,Government",en,Internet infrastructure should be publicly owned.,"internet,technology,economics,government"
2681,57144,"Given reasonable incentives, are charities mor...",https://www.kialo.com/given-reasonable-incenti...,2022-05-05T10:36:30.795Z,https://www.kialo.com/explore/popular,Charities should replace social security.,"Economics,Politics,Charity,Religion,Philosophy...",en,Charities should replace social security.,"economics,politics,charity,religion,philosophy..."
2716,5909,The sales tax should be replaced with the inco...,https://www.kialo.com/the-sales-tax-should-be-...,2020-08-18T16:17:30.644Z,https://www.kialo.com/explore/popular,The sales tax should be replaced with the inco...,"Economics,Taxation,Money",en,The sales tax should be replaced with the inco...,"economics,taxation,money"
2736,9823,Should it be possible to buy and sell citizens...,https://www.kialo.com/should-it-be-possible-to...,2020-12-03T06:09:45.753Z,https://www.kialo.com/explore/popular,It should be possible to buy and sell national...,"Economics,Society,Politics,Nationalism",en,It should be possible to buy and sell national...,"economics,society,politics,nationalism"


time: 85.9 ms (started: 2023-05-13 14:20:43 +00:00)


In [ ]:
answers_df

In [ ]:
# meta_data_df[meta_data_df['tags_list_translated'].str.contains('econom') & (meta_data_df['language']=='en')].to_excel(f'/content/drive/MyDrive/Kialo vs ChatBots/Query Results/Classification/economic_claims_Kialo.xlsx', index=None) #.map(len).sum()
# meta_data_df[meta_data_df['tags_list_translated'].str.contains('politic|gender|feminism|law|government|democracy|law|justice|equality|environment') & (meta_data_df['language']=='en')].to_excel(f'/content/drive/MyDrive/Kialo vs ChatBots/Query Results/Classification/sociopolitical_claims_Kialo.xlsx', index=None)

time: 255 ms (started: 2023-05-05 21:19:27 +00:00)


In [ ]:
answers_df['answer']

0       The right to choose abortion is a controversia...
1       The question of whether the government should ...
2       The question of whether global overpopulation ...
3       The question of whether all public bathrooms s...
4       The question of whether homeschooling should b...
                              ...                        
2821    Культура отмены - это современная форма острак...
2822    Введение 4-х дневной рабочей недели может быть...
2823    There are pros and cons to both systems. A pri...
2824    Веганство - это образ жизни, который стремится...
2825    El pensamiento crítico es una habilidad import...
Name: answer, Length: 2826, dtype: object

time: 9.76 ms (started: 2023-05-05 20:57:57 +00:00)


In [ ]:
def chunk_arguments(row,subject_np):
    # List of delimiters to use for chunking
    row = str(row)
    row=re.sub("http\S+|www.\S+"," ",row)
    row=re.sub("\["," ",row)
    row=re.sub("\]"," ",row)
    row=re.sub("\("," ",row)
    row=re.sub("\)"," ",row)
    row = re.sub('^(\n)+','',row)
    row = re.sub('^(\n)$','',row)
    # row = re.sub('(?i)\.?.*?as an ai.*?\.','.',row)
    # row = re.sub('(?i)^(no|yes).*?\.','.',row)
    # row = re.sub('(?i)\.?.*?(()|()|(fit-?all)).*?\.','.',row)
    adverbs = ['however', 'moreover', 'nevertheless', 'on the other side','on the opposite side','on the other hand', 'additionally', 'therefore','while some', 'others', 'thus', 'furthermore', 'in addition','in conclusion','ultimately','to conclude']
    middle_adverbs = ['however', 'moreover', 'nevertheless', 'on the other hand', 'additionally', 'therefore']

    row = re.sub(f'(?i), ({"|".join(middle_adverbs)}),','',row) 
    # row = re.sub(f'(?i)({"|".join(adverbs)}),?','',row)
    # row = re.sub('(?i)on the .*? side,?','',row)

    delimiters = ['\n', '.', ';', '!']#, 'however', 'moreover', 'nevertheless', 'on the other hand']
    delimiters = delimiters + adverbs

    # Combine delimiters into a regular expression pattern
    pattern = '|'.join(map(re.escape, delimiters))

    # Split response into arguments using regular expression pattern
    arguments = re.split(pattern, row,flags=re.IGNORECASE)
    remove_if_contain = ['pros','cons','pro','advantages','disadvantages','the question of', 'con', 'both', 'controversial', 'complex', 'answer', 'as an ai', 'language model','personal','matter','i can','whether','preferences','arguments','for and against', 'pro and con', 'pros and cons']

    # Remove leading/trailing whitespace from each argument
    arguments_cleaned = []
    for arg in arguments:
      if not re.match(f'(?i).*\\b({"|".join(remove_if_contain)})\\b.*',arg):
        arg = arg.strip()
        arg = re.sub('(?i)arguments (for|against|in favor (of)?)','',arg)
        arg = re.sub('(?i)^.{0,20}(may|might|can)?.{0,20}(argue|claim|say|believe|think|suggest) (that)?','',arg)
        # arg = re.sub('(?i)^.{0,20}(have|had)?.{0,20}(argued|claimed|said|believed|thought|suggested) (that)?','',arg)
        # row = re.sub(f'(?i),?({"|".join(adverbs)}),?','',row)

        # arg = re.sub('(?i)it is important to\s?.*?\s?(that)?','',arg)
        arg = re.sub('also|while','',arg)
        arg = re.sub('^.*?:','',arg)
        arg = re.sub('^\s*,','',arg)
        arg = re.sub(',\s*$','',arg)
        arg = re.sub('^\d','',arg)
        arg = arg.strip()
        arg = re.sub('.*\?$','',arg)
        if not re.match('(?i)it is important\s?(to|for)?\s?.*?\s?(that)?',arg):
          arg = re.sub('(?i)^(it|he|she|this concept|this idea)',subject_np,arg)
        if len(arg)>40:
          arguments_cleaned.append(arg)

    # return ' <sep> '.join(arguments_cleaned)
    return arguments_cleaned

time: 9.89 ms (started: 2023-05-17 17:43:55 +00:00)


time: 10.8 ms (started: 2023-04-29 17:50:43 +00:00)


In [ ]:
answers_df_ = answers_df[answers_df['tags_list_translated'].str.contains('econom')]#.sample(20,random_state = 5)
answers_df_['subject_np'] = answers_df_['initial_claim'].progress_apply(lambda x: get_subject_np(x))
answers_df_['arguments_list'] = answers_df_.progress_apply(lambda x: chunk_arguments(x['answer_translated'],x['subject_np']),axis=1)
answers_df_[['initial_claim','subject_np','answer_translated','arguments_list']]

  0%|          | 0/211 [00:00<?, ?it/s]

<ipython-input-8-024c87a10d82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df_['subject_np'] = answers_df_['initial_claim'].progress_apply(lambda x: get_subject_np(x))


  0%|          | 0/211 [00:00<?, ?it/s]

<ipython-input-8-024c87a10d82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df_['arguments_list'] = answers_df_.progress_apply(lambda x: chunk_arguments(x['answer_translated'],x['subject_np']),axis=1)


,initial_claim,subject_np,answer_translated,arguments_list
1,The government should provide funding for arts...,The government,"As an AI language model, I do not have persona...",[Arts programs provide numerous benefits to so...
7,Governments should make an effort to reduce th...,Governments,"As an AI language model, I do not have persona...",[Governments is widely recognized that reducin...
13,Governments should provide a universal basic i...,Governments,"As an AI language model, I do not have persona...",[Universal basic income UBI is a concept tha...
32,Wealthy countries should provide citizens with...,Wealthy countries,"As an AI language model, I do not have persona...",[The idea of a universal basic income UBI ha...
45,Nuclear energy should be used to replace fossi...,energy Nuclear,"As an AI language model, I do not have persona...",[Nuclear energy produces very low carbon emiss...
...,...,...,...,...
2716,The sales tax should be replaced with the inco...,sales,"As an AI language model, I do not have persona...",[- Income tax is generally considered to be a ...
2736,It should be possible to buy and sell national...,It,"As an AI language model, I do not have persona...",[Nationality is a fundamental aspect of a pers...
2739,The rise of Right-Wing Populism is tied to eco...,rise,"As an AI language model, I do not have persona...",[the rise of right-wing populism is tied to ec...
2763,"Las criptomonedas, encabezadas como forma de v...",encabezadas,"As a business model, cryptocurrencies have gre...","[As a business model, cryptocurrencies have gr..."


time: 2.2 s (started: 2023-05-13 14:24:15 +00:00)


In [ ]:
# answers_df_ = answers_df[answers_df['tags_list_translated']]
answers_df['subject_np'] = answers_df['initial_claim'].progress_apply(lambda x: get_subject_np(x))
answers_df['arguments_list'] = answers_df.progress_apply(lambda x: chunk_arguments(x['answer_translated'],x['subject_np']),axis=1)
answers_df[['initial_claim','subject_np','answer_translated','arguments_list']]

  0%|          | 0/2826 [00:00<?, ?it/s]

  0%|          | 0/2826 [00:00<?, ?it/s]

,initial_claim,subject_np,answer_translated,arguments_list
0,Pregnant people should have the right to choos...,Pregnant people,"As an AI language model, I do not have persona...",[The decision to have an abortion should be ma...
1,The government should provide funding for arts...,The government,"As an AI language model, I do not have persona...",[Arts programs provide numerous benefits to so...
2,Global overpopulation is a myth.,Global overpopulation,"As an AI language model, I do not have persona...",[the world's population is growing at an unsus...
3,All public bathrooms should be gender neutral.,All public bathrooms,"As an AI language model, I do not have persona...",[The idea of gender-neutral bathrooms is based...
4,Homeschooling should be banned.,Homeschooling,"As an AI language model, I do not have persona...",[Homeschooled children may not have the same o...
...,...,...,...,...
2821,Культура отмены - путь к прогрессу,отмены,Cancellation culture is a concept that describ...,[Cancellation culture is a concept that descri...
2822,Насколько целесообразна идея ввести 4-х дневну...,Насколько целесообразна,"As an AI, I cannot offer a personal opinion, b...","[Employees will have more time for family, fri..."
2823,A private money system with real bills and gol...,A private money system with real bills and gol...,"As an AI language model, I do not have persona...",[Advocates of a private money system with real...
2824,Веганство:за и против,против,Behind:\n\n1. Ethical considerations: Veganism...,[Veganism is based on the belief that animals ...


time: 29.9 s (started: 2023-05-17 17:44:00 +00:00)


In [ ]:
answers_df

In [ ]:
model = 'BingAI'
answers_df[answers_df['tags_list_translated'].str.contains('econom') & (answers_df['language']=='en')].explode('arguments_list').to_excel(f'.../Datasets/Query-Results/Classification/economic_arguments_AI_{model}.xlsx', index=None) #.map(len).sum()
# answers_df[answers_df['tags_list_translated'].str.contains('politic|gender|feminism|law|government|democracy|law|justice|equality|environment') & (answers_df['language']=='en')].explode('arguments_list').to_excel(f'/content/drive/MyDrive/Kialo vs ChatBots/Query Results/Classification/sociopolitical_arguments_AI_{model}.xlsx', index=None) #.map(len).sum()

time: 285 ms (started: 2023-05-05 21:22:38 +00:00)


In [ ]:
answers_df.explode('arguments_list')

,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated,subject_np,arguments_list
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1680106015,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,The decision to have an abortion should be mad...
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1680106015,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,It is important to respect the autonomy and ag...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Arts programs provide numerous benefits to soc...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Many arts programs are not financially sustain...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,The government has a responsibility to support...
...,...,...,...,...,...,...,...,...,...,...,...,...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,Encouraging research and analysis of sources a...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,Fostering teamwork allows students to explore ...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,By strengthening critical thinking in the educ...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,"E Manuela Beltrán, the civic skills and compet..."


time: 98.8 ms (started: 2023-05-17 17:46:41 +00:00)


In [ ]:
answers_df.explode('arguments_list').to_excel(f'.../Datasets/Query-Results/Classification/all_arguments_AI_{model}.xlsx', index=None) #.map(len).sum()

time: 4.71 s (started: 2023-05-17 17:46:24 +00:00)


In [ ]:
answers_df[answers_df['tags_list_translated'].str.contains('politic|gender|feminism|law|government|democracy|law|justice|equality|environment') & (answers_df['language']=='en')].explode('arguments_list')

,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated,subject_np,arguments_list
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1680106015,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,The decision to have an abortion should be mad...
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1680106015,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,It is important to respect the autonomy and ag...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Arts programs provide numerous benefits to soc...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Many arts programs are not financially sustain...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,The government has a responsibility to support...
...,...,...,...,...,...,...,...,...,...,...,...,...
2816,59609,https://www.kialo.com/should-we-have-a-global-...,We should have a global governance.,gpt-3.5-turbo,We should have a global governance?,"As an AI language model, I do not have persona...",1680289164,en,"As an AI language model, I do not have persona...","governance,political,decisionmaking",We,nations should be free to make their own decis...
2819,60257,https://www.kialo.com/knowledgeunderstanding--...,Desire and emotions are more important than fa...,gpt-3.5-turbo,Desire and emotions are more important than fa...,"As an AI language model, I do not have persona...",1680289217,en,"As an AI language model, I do not have persona...","knowledge,emotion,logic,politics,philosophy,so...",Desire and emotions,Desire and emotions can be powerful motivators...
2819,60257,https://www.kialo.com/knowledgeunderstanding--...,Desire and emotions are more important than fa...,gpt-3.5-turbo,Desire and emotions are more important than fa...,"As an AI language model, I do not have persona...",1680289217,en,"As an AI language model, I do not have persona...","knowledge,emotion,logic,politics,philosophy,so...",Desire and emotions,Facts and logic provide a more objective and r...
2819,60257,https://www.kialo.com/knowledgeunderstanding--...,Desire and emotions are more important than fa...,gpt-3.5-turbo,Desire and emotions are more important than fa...,"As an AI language model, I do not have persona...",1680289217,en,"As an AI language model, I do not have persona...","knowledge,emotion,logic,politics,philosophy,so...",Desire and emotions,knowledge and understanding can help individua...


time: 40.8 ms (started: 2023-05-05 17:14:20 +00:00)


# Chunk Pros and Cons Format

In [4]:
models = ['text-curie-001','text-babbage-001','text-davinci-001','text-davinci-002','text-davinci-003','gpt-3.5-turbo']
query_type = 'questionmark'
query_type = 'proscons'
model = 'gpt-3.5-turbo'
# model = 'BingAI'
# model = 'text-davinci-002'
meta_data_df = pd.read_excel('/content/drive/MyDrive/Kialo vs ChatBots/Meta_Data/Kialo_MetaData_popular.xlsx')

query_file_path = f'/content/drive/MyDrive/Kialo vs ChatBots/Query Results/Kialo - Open AI/kialo_{query_type}_{model}.csv'
# query_file_path = f'/content/drive/MyDrive/Kialo vs ChatBots/Query Results/Kialo - Bing AI/Bing AI Queries Merged - corrected.xlsx'

answers_df = pd.read_csv(query_file_path)
# answers_df = pd.read_excel(query_file_path)

answers_df['tags_list_translated']=meta_data_df['tags_list_translated'].copy()



time: 3.72 s (started: 2023-05-24 19:02:42 +00:00)


In [5]:
answers_df

,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Women have the right to control thei...,1684680193,en,Pros:\n1. Women have the right to control thei...,"feminism,abortion,health,religion,women,law"
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Encourages creativity and self-expre...,1684680224,en,Pros:\n1. Encourages creativity and self-expre...,"art,government,economics,education"
2,9226,https://www.kialo.com/global-overpopulation-is...,Global overpopulation is a myth.,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. It challenges the commonly held beli...,1684680253,en,Pros:\n1. It challenges the commonly held beli...,"society,humans,earth,environment,climatechange"
3,1266,https://www.kialo.com/gender-neutral-bathrooms...,All public bathrooms should be gender neutral.,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Inclusivity: Gender-neutral bathroom...,1684680278,en,Pros:\n1. Inclusivity: Gender-neutral bathroom...,"gender,feminism,transgender,sex,equality"
4,9647,https://www.kialo.com/should-homeschooling-be-...,Homeschooling should be banned.,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Homeschooling may not provide childr...,1684680305,en,Pros:\n1. Homeschooling may not provide childr...,"education,policy,usa,law,school,america"
...,...,...,...,...,...,...,...,...,...,...
2821,60719,https://www.kialo.com/%D0%BA%D1%83%D0%BB%D1%8C...,Культура отмены - путь к прогрессу,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Encourages accountability and respon...,1684776746,ru,Pros:\n1. Encourages accountability and respon...,culture of cancellation
2822,61035,https://www.kialo.com/%D0%BD%D1%83%D0%B6%D0%BD...,Насколько целесообразна идея ввести 4-х дневну...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Improved work-life balance: A four-d...,1684776770,ru,Pros:\n1. Improved work-life balance: A four-d...,work
2823,61059,https://www.kialo.com/a-private-money-system-w...,A private money system with real bills and gol...,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Stability: A private money system wi...,1684776797,en,Pros:\n1. Stability: A private money system wi...,money
2824,61106,https://www.kialo.com/%D0%B2%D0%B5%D0%B3%D0%B0...,Веганство:за и против,gpt-3.5-turbo,Provide pros and cons for the following statem...,Pros:\n1. Ethical treatment of animals: Vegans...,1684776825,mk,Pros:\n1. Ethical treatment of animals: Vegans...,veganism


time: 26.8 ms (started: 2023-05-24 19:02:49 +00:00)


In [ ]:
def get_unique_index(lst, value):
    if lst.count(value) == 1:
        return lst.index(value)
    else:
        raise ValueError("Value is not unique in the list")

def clean_arguments(arguments):
    arguments_cleaned = []
    for arg in arguments:
        arg = arg.strip()
        arg = re.sub('^\d\.?','',arg)
        arg = re.sub('^-','',arg)
        arg = arg.strip()
        if len(arg)>3:
          arguments_cleaned.append(arg)
    return arguments_cleaned

def chunk_arguments(row):
    # List of delimiters to use for chunking
    row = str(row)

    delimiters = ['\n']#, 'however', 'moreover', 'nevertheless', 'on the other hand']

    # Combine delimiters into a regular expression pattern
    pattern = '|'.join(map(re.escape, delimiters))

    # Split response into arguments using regular expression pattern
    arguments = re.split(pattern, row,flags=re.IGNORECASE)
    try:
      pros_index = get_unique_index(arguments,'Pros:')
      cons_index = get_unique_index(arguments,'Cons:')
      pros = clean_arguments(arguments[(pros_index+1):(cons_index)])
      cons = clean_arguments(arguments[(cons_index+1):])
      stances = ['Pro'] * len(pros) + ['Con'] * len(cons)
      return pd.Series([pros+cons,stances])
    except:
      arguments = clean_arguments(arguments)
      stances = ['Unanswered'] * len(arguments)
      return pd.Series([arguments,stances])

time: 1.52 ms (started: 2023-05-22 18:50:36 +00:00)


time: 10.8 ms (started: 2023-04-29 17:50:43 +00:00)


In [ ]:
# answers_df_ = answers_df[answers_df['tags_list_translated']]
# answers_df['subject_np'] = answers_df['initial_claim'].progress_apply(lambda x: get_subject_np(x))
answers_df[['arguments_list','stance_given']] = answers_df.progress_apply(lambda x: chunk_arguments(x['answer_translated']),axis=1)
answers_df[['initial_claim','subject_np','answer_translated','arguments_list','stance_given']].head(5)

  0%|          | 0/2826 [00:00<?, ?it/s]

,initial_claim,subject_np,answer_translated,arguments_list,stance_given
0,Pregnant people should have the right to choos...,Pregnant people,Pros:\n1. Women have the right to control thei...,[Women have the right to control their own bod...,"[Pro, Pro, Pro, Pro, Pro, Con, Con, Con, Con, ..."
1,The government should provide funding for arts...,The government,Pros:\n1. Encourages creativity and self-expre...,[Encourages creativity and self-expression: Ar...,"[Pro, Pro, Pro, Pro, Con, Con, Con, Con]"
2,Global overpopulation is a myth.,Global overpopulation,Pros:\n1. It challenges the commonly held beli...,[It challenges the commonly held belief that t...,"[Pro, Pro, Pro, Con, Con, Con]"
3,All public bathrooms should be gender neutral.,All public bathrooms,Pros:\n1. Inclusivity: Gender-neutral bathroom...,[Inclusivity: Gender-neutral bathrooms provide...,"[Pro, Pro, Pro, Pro, Con, Con, Con, Con]"
4,Homeschooling should be banned.,Homeschooling,Pros:\n1. Homeschooling may not provide childr...,[Homeschooling may not provide children with t...,"[Pro, Pro, Pro, Pro, Pro, Con, Con, Con, Con, ..."


time: 782 ms (started: 2023-05-22 18:50:38 +00:00)


In [ ]:
answers_df_exploded = answers_df.explode('arguments_list')
answers_df_exploded['stance_given'] = answers_df.explode('stance_given')['stance_given']

time: 47.1 ms (started: 2023-05-22 18:58:27 +00:00)


In [ ]:
answers_df_exploded

In [ ]:
model = 'BingAI'
answers_df[answers_df['tags_list_translated'].str.contains('econom') & (answers_df['language']=='en')].explode('arguments_list').to_excel(f'.../Datasets/Query-Results/Classification/economic_arguments_AI_{model}.xlsx', index=None) #.map(len).sum()
# answers_df[answers_df['tags_list_translated'].str.contains('politic|gender|feminism|law|government|democracy|law|justice|equality|environment') & (answers_df['language']=='en')].explode('arguments_list').to_excel(f'.../Datasets/Query-Results/Classification/sociopolitical_arguments_AI_{model}.xlsx', index=None) #.map(len).sum()

time: 285 ms (started: 2023-05-05 21:22:38 +00:00)


In [ ]:
answers_df.explode('arguments_list')

,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated,subject_np,arguments_list
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1680106015,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,The decision to have an abortion should be mad...
0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1680106015,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,It is important to respect the autonomy and ag...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Arts programs provide numerous benefits to soc...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Many arts programs are not financially sustain...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,The government has a responsibility to support...
...,...,...,...,...,...,...,...,...,...,...,...,...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,Encouraging research and analysis of sources a...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,Fostering teamwork allows students to explore ...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,By strengthening critical thinking in the educ...
2825,61071,https://www.kialo.com/debatiendo-la-escuela-%F...,Desarrollar el pensamiento crítico en los proc...,gpt-3.5-turbo,Desarrollar el pensamiento crítico en los proc...,El pensamiento crítico es una habilidad fundam...,1680289394,es,Critical thinking is a fundamental skill for t...,"tics,diversity,inclusion,teaching,leadership,c...",Desarrollar el pensamiento crítico en los proc...,"E Manuela Beltrán, the civic skills and compet..."


time: 98.8 ms (started: 2023-05-17 17:46:41 +00:00)


In [ ]:
answers_df_exploded.to_excel(f'.../Datasets/Query-Results/Classification/all_arguments_AI_{query_type}_{model}.xlsx', index=None) #.map(len).sum()


time: 8.35 s (started: 2023-05-22 19:03:48 +00:00)


In [ ]:
answers_df_exploded